## QCoDeS Training for Developers

Welcome to the training about QCoDeS targeted for developers. After this training, developers should have more knowledge about QCoDeS which will bring them confidence with contributing to it.

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

### Approach

- Don't bring too much relevant background, only small absolutely necessary portions
- Quickly/slightly refresh the user-side view on QCoDeS, and get close to code
- Describe features/concepts from ideas, but get close to code very quickly

### Execution plan for presenter

- Before:
  - Have qcodes code at hand in your IDE https://github.com/qcodes/qcodes
  - Run the training Jupyter notebook, and have it at hand https://github.com/astafan8/Qcodes/blob/training-for-devs/docs/examples/QCoDeS%20Training%20for%20Developers.ipynb
  - Familiarize yourself with the following info from the docs:
    - Front page https://qcodes.github.io/Qcodes/index.html
    - 15 minutes of QCoDeS https://qcodes.github.io/Qcodes/examples/15_minutes_to_QCoDeS.html
    - `Parameter`s
      - overview1 https://qcodes.github.io/Qcodes/examples/Parameters/Parameters.html
      - overview2 https://qcodes.github.io/Qcodes/examples/writing_drivers/Creating-Instrument-Drivers.html#What%E2%80%99s-a-Parameter? - only this section
      - API docs https://qcodes.github.io/Qcodes/api/parameters/parameter.html#module-qcodes.instrument.parameter
      - `ParameterWithSetpoints` https://qcodes.github.io/Qcodes/examples/Parameters/Simple-Example-of-ParameterWithSetpoints.html
    - Performing measurements https://qcodes.github.io/Qcodes/examples/DataSet/Performing-measurements-using-qcodes-parameters-and-dataset.html
    - dataset notebooks
      - the class https://qcodes.github.io/Qcodes/examples/DataSet/DataSet-class-walkthrough.html
      - data access https://qcodes.github.io/Qcodes/examples/DataSet/Accessing-data-in-DataSet.html
- During:
  - Go through notebook in 3 parts, have 5 min breaks, 5 min reserve beginning, 10 min reserve at the end
  - Have a time keeper
  - Let ask questions at specific moments but very frequently to feel interactive
- After:
  - Port notebook to qcodes docs ASAP
  - Sessions on some parts if requested

### Imports

In [1]:
import qcodes

from typing import Optional, Sequence, Dict
from pprint import pprint

import numpy

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\a-miasta\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\a-miasta\.qcodes\logs\200414-26956-qcodes.log


## QCoDeS? ah? [5min]

What is it? What are it's goals, and users? And why?

### "The It"

**Q.** uantum **Co** penhagen **De** lft **S** ydney:

- data acquisition framework
- modular
- python-based
- open-source

### Goal

**Common** framework for physics experiments:

- users need to write only their-own-experiment-specific code
- physics experiments can take advantage of modern software and best practices
- code can and should be contributed back to the framework
- the process of moving between teams or labs, and of setting up a new experiment is streamlined
- new students don’t need to spend a long time learning software in order to participate in experiments

### Users

- people doing measurements:
  - academics: e.g. scientists and students
  - corporates: e.g. experimentalists, engineers, hardware manufacturers
- worldwide

### Driven by

- Microsoft, largely
- worldwide comminity, growing

### Development principles

- Robust code - as opposed to what users are used to from other measurement frameworks
- Respect backwards compatibility - don't break user's code that "works"!
- Modularity - extend-existing, not go-off-and-write-your-own
- User facing API - list in docs, devs can use everything as appropriate
- All of the above is basically maintatining users' trust
- Carefully scoping problems

## I wanna MEASURE [30min]

How do I measure things? How are measurements enabled via QCoDeS?

We will be looking into the following pyramid of objects and concepts:
- Station - the ultimate bucket
  - Instruments - and channels
    - Parameters - many kinds

### Station

``Station`` is just a bucket for components such as instruments, parameters, etc.

Speaking more strictly, those components should be snapshot-able (subclasses of ``Metadatable``), but more about this in the section about the snapshot.

``Station`` can be created with python code and/or using YAML configuration file. [Read on Station, its interface, YAML config, and more here](http://qcodes.github.io/Qcodes/examples/Station.html).

In [2]:
from qcodes.station import Station

#### Default station

Notice the ``default=True`` argument of the ``Station`` - it controls wether to store the initialized ``Station`` object into an attribute.

Using ``Station.default`` as a fallback in other objects in QCoDeS is very common, as it is extremely uncommon that more than one ``Station`` is initiated in one python session.

In [3]:
station = Station()  # default=True

assert Station.default is station

station2 = Station(default=False)

assert station2 is not Station.default

### Instruments

Instrument conceptually represents hardware that one needs to talk to in order to perform measurements.

The concept can be expanded to also include "instruments" that are:

- not remotly controllable but it's important to record their state
- exposing a more measurement-specific interface while aggregating actual instruments

#### Drivers

Concrete implementation of classes and abstracations provided by QCoDeS for particular piece of hardware is commonly referred to as ``drivers`` or ``QCoDeS drivers`` for that piece of hardware.

From a real conversation:

> - Shall we buy this useful, powerful, and expensive measurement equipment?
> - Hmm... Is there a QCoDeS driver for it?

#### Base class(es)

From coding perspective, instruments in QCoDeS are subclasses of ``Instrument`` and ``InstrumentBase`` from ``qcodes.instrument.base``.

``InstrumentBase`` - defines interface:

- has a name
- is snapshot-able (subclass of ``Metadatable``)
- serves as a bucket for ``Parameter``s
- may include snapshot-able "submodules"
  - navigation via ``root_instrument``, ``parent``, ``ancestors``
- instance-bound logger
- convenient dot-notation access to it's parameters and submodules (thanks to ``DelegateAttributes``)
- convenient ``[]``-notation access to them as well

``Instrument`` - accounts for communication:

- subclass of ``InstrumentBase``
- hides communication with hardware via ``ask``/``write``
- implements global registry of instrument instances
- ``IDN``, ``connect_message``

#### Submodules

Submodules allow for logical grouping of functionality within an instrument. For example, [look at the trigger, sample, and display submodules of Keysight 344xxA driver](https://qcodes.github.io/Qcodes/examples/driver_examples/Qcodes%20example%20with%20Keysight%20344xxA.html#Parameters-and-methods-overview) which allow to conveniently group parameters and methods without polluting the root namespace of the driver.

To add a submodule to an instrument, use it's ``add_submodule`` method (usually in instrument's ``__init__``).

``InstrumentBase`` promises to include snapshots of it's submodules in its own snapshot.

How to implement submodules may not be clear:
- the current implementation suggests that any ``Metadatable`` object will work
- the typing of ``add_submodule`` suggests using/subclassing ``InstrumentBase`` in order to take full advantage of the ``InstrumentBase``s interface for the submodule
- most commonly used/cubclasses is ``InstrumentChannel`` from ``qcodes.instrument.channel`` because
  - it takes care of correct naming and 
  - forwards communication to the parent instrument

#### Channels

Channels are most popular submodules of instruments. Why? Think of a multi-channel voltage source kind of instrument.

QCoDeS provides ``InstrumentChannel`` class that takes care of correct naming and forwards communication to the parent instrument. ``InstrumentChannel`` can be added as submodules to the instrument.

Moreover, QCoDeS provides ``ChannelList`` class. In short, it allows to group channels that are of the same class, and implements convenient access to common parameters and functions of those channels. See the difference between the following ways of setting voltage parameter of all channels to ``0``:
```python
my_voltage_source.channels.voltage(0)
```
versus
```python
for channel in my_voltage_source.channels:
    channel.voltage(0)
```
or even worse
```python
channel_names = [...]  # I just happen to know them
for channel_name in channel_names:
    channel = my_voltage_source[channel_name]
    channel.voltage(0)
```

#### Hardware communication

``Instrument`` class suggests the most frequent use case of how communication with hardware is being performed:
- ``write(cmd: str) -> None``
- ``ask(cmd: str) -> str``

Subclasses of ``Instrument`` should implement concrete communication by overriding ``write_raw`` and ``ask_raw``.

A lot of QCoDeS abstractions are built with this string-command-based communication model in mind, especially ``Parameter``.

##### ``VisaInstrument``

A great example is ``VisaInstrument`` from ``qcodes.instrument.visa`` - it implements ``write_raw`` and ``ask_raw`` on top of ``pyvisa`` library that provides a python wrapper over implementations of VISA (Virtual Instrument Software Architecture) API.

Here's a schematic of the layering, from high, user-facing, to low:
```
- MyInstrumentDriver . parameter/method
- VisaInstrument     . ask/write
- Instrument         . ask_raw/write_raw
- pyvisa .. Resource . query/write
- Actual hardware communication layer, e.g. LAN, USB
  implemented by a VISA driver (e.g. National Instruments, pyvisa-py, etc)
```

The command API of the most VISA instruments conforms to SCPI (Standard Commands for Programmable Instruments) standard which makes QCoDeS driver development for those instruments extremely easy.

##### Other

QCoDeS does **not** restrict interacting with hardware only with this communication model. An good example of this are instruments which use some specific ``.dll`` library, see drivers for [Alazar high-speed digital acquisition cards](https://github.com/QCoDeS/Qcodes/blob/master/qcodes/instrument_drivers/AlazarTech/ATS.py#L68), [Vaunix step attenuator](https://github.com/QCoDeS/Qcodes_contrib_drivers/blob/c007532d48dfc035a9e2f03167606c16c42d7f77/qcodes_contrib_drivers/drivers/Vaunix/LDA.py#L59), etc.

In that case, ``write_raw``/``ask_raw`` are usually left as raising ``NotImplementedError``, and parameters and methods of the instrument driver don't use them at all.

#### Registry of instances

QCoDeS does not allow creation of an instrument with the same ``name``. Why? - To enforce that there is **only one** connection to an instrument.

##### No instruments with same name

Let's try to create two instruments with the same name:

In [4]:
from qcodes.instrument.base import Instrument

foo_called_foo = Instrument('foo')

In [5]:
try:
    bar_called_foo = Instrument('foo')
except:
    import traceback
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-5-1e60ed53d4d7>", line 2, in <module>
    bar_called_foo = Instrument('foo')
  File "c:\users\a-miasta\pycharmprojects\qcodes\qcodes\instrument\base.py", line 431, in __init__
    self.record_instance(self)
  File "c:\users\a-miasta\pycharmprojects\qcodes\qcodes\instrument\base.py", line 567, in record_instance
    raise KeyError('Another instrument has the name: {}'.format(name))
KeyError: 'Another instrument has the name: foo'


In order to get rid of the exception, the user needs to ``close`` one of the instruments:

In [6]:
foo_called_foo.close()

In [7]:
bar_called_foo = Instrument('foo')
bar_called_foo

<Instrument: foo>

##### Fool proof?

This may not be the optimal mechanism.

For example, one can fool the mechanism by using two different names but the same VISA address:
```python
foo_at_address1 = VisaInstrument('foo', address='address1')
bar_at_address1 = VisaInstrument('bar', address='address2')
# DISCLAIMER: only tried with pyvisa-sim, not a real VISA driver
```
in which case it's now up to the "connectivity backend" (in this case, VISA) to deal with having multiple "connections" to the same piece of hardware.

##### Behind the scenes

Behind the scenes QCoDeS maintains a registry of instrument instances by their names. The scope of the registry is the python process.

``Instrument`` class maintains an ``_all_instruments`` name-to-weak-reference dictionary in a class attribute.

Each ``Instrument`` subclass maintains its own ``_instances`` list of weak references to instances of **only that (sub)class** as in a class attribute (and as ``instances`` method).

``Instrument`` class provides a few class methods for navigating in the registry: ``instances``, ``find_instrument``, ``exists``, ``is_valid``. The most popular however are ``close_all`` method and ``find_or_create-instrument`` function.

##### ``find_or_create_instrument``

There is a ``find_or_create_instrument`` convenience function that allows to conveniently find an existing instrument, create it if it doesn't exists, or re-create it if it does exist:

In [8]:
from qcodes.instrument.base import find_or_create_instrument

In [9]:
# Example of Find
foo = find_or_create_instrument(Instrument, 'foo')
assert foo is bar_called_foo

Connected to: None foo (serial:None, firmware:None) in 0.04s


In [10]:
# Example of Recreate
foo = find_or_create_instrument(Instrument, 'foo', recreate=True)
assert foo is not bar_called_foo

In [11]:
# Example of Create
class MyInstrument(Instrument):
    pass

foo = find_or_create_instrument(MyInstrument, 'bar')

##### ``close_all``

Another useful API is ``close_all`` - it closes all instrument instances.

In [12]:
print('before closing all:', Instrument._all_instruments)

Instrument.close_all()

print('after closing all:', Instrument._all_instruments)

before closing all: {'foo': <weakref at 0x00000263988B01D8; to 'Instrument' at 0x0000026398899448>, 'bar': <weakref at 0x00000263988CDEA8; to 'MyInstrument' at 0x000002639566C1C8>}
after closing all: {}


##### And ``Station``?

``Station`` is aware of the instrument instance registry. It exposes convenient API which is station-scoped version of the above-mentioned ``Instrument`` APIs:

- ``.load_instrument(.., revive_instance=False, ..)`` - for ``find_or_create_instrument``
- ``.close_and_remove_instrument(..)`` - for ``Instrument.close``
- ``.close_all_registered_instruments(..)`` - for ``Instrument.close_all``

### Parameters

Parameters in QCoDeS serve mostly two purposes:

- values that can be set or measured via an instrument, e.g. voltage, current, time trace
- "settings" of an instrument, e.g. range, precision, NPLCs

	• Parameters
		○ What they are - two purposes
		○ Name/label/unit/docstring - metadata
		○ Set/get
			§ _raw
			§ command
		○ instrument's add_parameter         
		○ Set_to
		○ Cache and get_latest
		○ Value/raw_value
			§ Parsers
			§ Val_mapping
			§ Validators
		○ Snapshot - it's nontriviallity and flags
		○ More specific parameters:
			§ Delegate parameter
			§ Array + setpoints
			§ Multiparameter
			§ Group parameter
			§ How to subclass in general

### Snapshot

Snapshot is a "photograph" of a state of a measurement setup or its piece, be it ``Station``, ``Instrument``, ``Parameter``, or other components.

Snapshot is **not** maintained in memory. Instead, each snapshot-able component can always return/produce its snapshot.

#### Subclass ``Metadatable``

Snapshot-able objects should inherit from ``qcodes.utils.metadata.Metadatable`` and should override ``snapshot_base`` method to return a dictionary representing the state of the object. Then, calling ``snapshot()`` method will return the snapshot of the object.

In [13]:
from qcodes.utils.metadata import Metadatable


class TeamsStatus(Metadatable):
    def __init__(self, name: str, status: str, metadata=None):
        super().__init__(metadata=metadata)
        self.name = name
        self.status = status

    def snapshot_base(
            self, update: bool = False,
            params_to_skip_update: Optional[Sequence[str]] = None) -> Dict:
        return {'status': self.status}


teams_status = TeamsStatus('Fridge25 status', 'busy')

pprint(teams_status.snapshot())

{'status': 'busy'}


####  ``metadata`` argument

Note that ``Metadatable`` also implements ``metadata`` keyword argument - ``metadata`` is expected to be a dictionary, and it will be included in the snapshot automatically (if it is not ``{}``).

In [14]:
favorites = Metadatable(metadata={'number': 6 + 9j})
favorites.load_metadata({'fruit': 'manadrine'})  # ``dict.update`` behavior

In [15]:
pprint(favorites.snapshot())

{'metadata': {'fruit': 'manadrine', 'number': (6+9j)}}


#### Add  ``Metadatable`` to station

``Metadatable`` objects can be added to the ``Station``, other objects - should not (otherwise ``snapshot()`` will raise).

In [16]:
station = Station(teams_status)
station.add_component(favorites)

'component1'

In [17]:
pprint(station.snapshot())

{'components': {'Fridge25 status': {'status': 'busy'},
                'component1': {'metadata': {'fruit': 'manadrine',
                                            'number': (6+9j)}}},
 'config': None,
 'default_measurement': [],
 'instruments': {},
 'parameters': {}}


#### Create ``JSON`` snapshot

Although snapshot is a python dictionary, it is always meant to be converted to JSON and stored next to the measured data. QCoDeS maintains it's own JSON converter class, ``qcodes.utils.helpers.NumpyJSONEncoder``, to support conversion of objects that are frequently met in measurement world, such as ``numpy`` arrays, complex numbers, pickle-ables.

In [18]:
from qcodes.utils.helpers import NumpyJSONEncoder
import json

json_snapshot = json.dumps(
    station.snapshot(), cls=NumpyJSONEncoder,
    indent=2, sort_keys=True  # for pretty-printing
)

In [19]:
print(json_snapshot)

{
  "components": {
    "Fridge25 status": {
      "status": "busy"
    },
    "component1": {
      "metadata": {
        "fruit": "manadrine",
        "number": {
          "__dtype__": "complex",
          "im": 9.0,
          "re": 6.0
        }
      }
    }
  },
  "config": null,
  "default_measurement": [],
  "instruments": {},
  "parameters": {}
}


#### ``update`` arguments

Note that ``snapshot`` has an ``update`` kwarg, and ``snapshot_base`` has ``update`` and ``params_to_skip_update`` kwargs - those allow to optimize querying the state of a ``Metadatable`` object.

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

## BREAK [5min]

## I wanna STORE [30min]

How and where do I store what I measured? And access back? Database, experiments, datasets, parameters?

- Quickly DB structure, paramtypes, data saving approach
- mention DataSet is too close to sqlite
- dig deeper into how Measurement/DataSet are designed

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)


	• How people measure - interactively!
		○ Why loop and not loop anymore
		○ Measurement as general thing and create-your-own-measurement-function
		○ Dond, pysweep, etc.
	• Measurement -> DataSet
		○ ASSUME THAT 'how to use them' IS KNOWN
		○ Name, experiment, station
		○ Parameters
			§ Register_* - as interface between qcodes parameters and dataset parameters
			§ Rundescription/interdeps/paramspecbase
			§ paramtype
		○ Before and after actions :)
		○ .run() -> Runner -> DataSaver - what is their role?
			§ Runner
			§ DataSaver
				□ Add_results
					® Validation that happens
				□ Dataset/run_id
		○ Sqlite data saving optimizations
			§ Paramtype "array"
			§ wrtie_period
			§ write_in_background
		○ Subscribers - bonus topic
	• Database stuff
		○ Runs table as log of performed measurements
		○ Experiment as a bucket of datasets but not used as such really
			§ Ideas for "active experiment, etc"
		○ Data storage
			§ As a message queue/log - needs reconstruction
			§ in the same DB
		○ Sqlite database, schema, versions
			§ Connect - users might forget to close, etc
		○ Sqlite module overview
	• DataSet walkthrough
		○ Identity, also timestamps and state
		○ parameters
		○ Getting data
		○ Metadata feature


## BREAK [5min]

## I want MORE [5min]

What else does QCoDeS provide out of the box?

- q/a, other qcodes topics for self-exploration

![qcodes-logo](https://qcodes.github.io/Qcodes/_images/qcodes_logo.png)

- contrib-drivers
- Config
- DelegateAttributes
- instrument functions
- Plotting support
- Instrument's functions
- Deprecation
- Linking datasets
- Extracting runs/datasets from db to another
- Monitor
- Interactive widget
- Legacy, loop-entangled, features
- writing drivers

## Presenter-only section

Some info, notes, and scratch-space for presenter.

qcodes logo means that the section(s) under it's level is(are) not ready.